In [7]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import zipfile
import os

# Step 1: Extract and Load Datasets
# Unzipping the dataset
zip_file_path = "Property_data-20250109T061007Z-001.zip"
extraction_dir = "property_data_extracted"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)

# List the extracted files
property_data_folder = os.path.join(extraction_dir, "Property_data")
property_data_files = os.listdir(property_data_folder)

In [9]:
# Merging the datasets
merged_dataframes = []
for file_name in property_data_files:
    file_path = os.path.join(property_data_folder, file_name)
    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')
        merged_dataframes.append(df)
    except Exception as e:
        print(f"Error reading {file_name}: {e}")

merged_properties_df = pd.concat(merged_dataframes, ignore_index=True)

# Step 2: Process property_photos.tsv
property_photos_path = "property_photos.tsv"
property_photos_df = pd.read_csv(property_photos_path, sep="\t")

def extract_photo_count(photo_urls):
    if pd.isna(photo_urls):
        return 0
    try:
        return len(eval(photo_urls))
    except Exception:
        return 0

property_photos_df['photo_count'] = property_photos_df['photo_urls'].apply(extract_photo_count)
property_photos_df = property_photos_df.drop(columns=['photo_urls'])
merged_properties_df = merged_properties_df.merge(property_photos_df, on="property_id", how="left")
merged_properties_df['photo_count'] = merged_properties_df['photo_count'].fillna(0).astype(int)

# Step 3: Process property_interactions.csv
property_interactions_path = "property_interactions.csv"
property_interactions_df = pd.read_csv(property_interactions_path)
property_interactions_df['request_date'] = pd.to_datetime(property_interactions_df['request_date'], errors='coerce', dayfirst=True)
merged_properties_df['activation_date'] = pd.to_datetime(merged_properties_df['activation_date'], errors='coerce', dayfirst=True)

interaction_with_activation = property_interactions_df.merge(
    merged_properties_df[['property_id', 'activation_date']],
    on='property_id',
    how='inner'
)
interaction_with_activation['days_since_activation'] = (
    interaction_with_activation['request_date'] - interaction_with_activation['activation_date']
).dt.days

In [10]:
# QUESTION 9: Locality with the highest average rent
highest_avg_rent_locality = merged_properties_df.groupby('locality')['rent'].mean().idxmax()
print("Q9: Locality with the highest average rent:", highest_avg_rent_locality)

Q9: Locality with the highest average rent: Bellandur


In [11]:
# QUESTION 10: Feature with the highest correlation with rent
correlation_features = merged_properties_df[['bathroom', 'property_age', 'property_size', 'deposit', 'rent']].corr()
highest_corr_feature = correlation_features['rent'].drop('rent').idxmax()
print("Q10: Feature with the highest correlation with rent:", highest_corr_feature)

Q10: Feature with the highest correlation with rent: bathroom


In [12]:

# QUESTION 11: Total interactions received within 7 days of activation
interactions_within_7_days = interaction_with_activation[interaction_with_activation['days_since_activation'] <= 7]
interactions_count = interactions_within_7_days['property_id'].value_counts().mode()[0]
print("Q11: Total interactions within 7 days:", interactions_count)

Q11: Total interactions within 7 days: 1


In [13]:
# QUESTION 12: Most frequent property age category
def categorize_property_age(age):
    if age <= 1:
        return "New"
    elif age <= 5:
        return "Less than 5 years"
    elif age <= 10:
        return "5 to 10 years"
    elif age <= 20:
        return "10 to 20 years"
    else:
        return "More than 20 years"

merged_properties_df['property_age_category'] = merged_properties_df['property_age'].apply(categorize_property_age)
most_frequent_age_category = merged_properties_df['property_age_category'].mode()[0]
print("Q12: Most frequent property age category:", most_frequent_age_category)


Q12: Most frequent property age category: New


In [22]:
# Calculate total_interactions for each property
interaction_counts = property_interactions_df['property_id'].value_counts().reset_index()
interaction_counts.columns = ['property_id', 'total_interactions']
merged_properties_df = merged_properties_df.merge(interaction_counts, on='property_id', how='left')
merged_properties_df['total_interactions'] = merged_properties_df['total_interactions'].fillna(0).astype(int)

In [23]:
# QUESTION 13: Apartment type with highest average interactions
highest_interaction_type = merged_properties_df.groupby('type')['total_interactions'].mean().idxmax()
print("Q13: Apartment type with highest average interactions:", highest_interaction_type)



Q13: Apartment type with highest average interactions: RK1


In [15]:
# QUESTION 14: Amenity with the greatest impact on rent
for amenity in ['gym', 'lift', 'swimming_pool']:
    merged_properties_df[amenity] = pd.to_numeric(merged_properties_df[amenity], errors='coerce')
amenity_correlation = {
    amenity: merged_properties_df[['rent', amenity]].corr().loc['rent', amenity]
    for amenity in ['gym', 'lift', 'swimming_pool']
}
highest_impact_amenity = max(amenity_correlation, key=amenity_correlation.get)
print("Q14: Amenity with the greatest impact on rent:", highest_impact_amenity)



Q14: Amenity with the greatest impact on rent: lift


In [24]:
# QUESTION 15: Highest interaction locality among top 5 localities
localities_avg_rent = merged_properties_df.groupby('locality')['rent'].mean().sort_values(ascending=False).head(5)
highest_interaction_locality = merged_properties_df[
    merged_properties_df['locality'].isin(localities_avg_rent.index)
].groupby('locality')['total_interactions'].sum().idxmax()
print("Q15: Locality with highest interactions among top 5 localities:", highest_interaction_locality)



Q15: Locality with highest interactions among top 5 localities: Whitefield


In [25]:
# QUESTION 16: Photo count for the property with the highest interactions
highest_interactions_property = merged_properties_df['total_interactions'].idxmax()
photo_count_highest_interactions = merged_properties_df.loc[highest_interactions_property, 'photo_count']
print("Q16: Photo count of property with highest interactions:", photo_count_highest_interactions)



Q16: Photo count of property with highest interactions: 0


In [26]:
# QUESTION 17: Hypothesis test for gym vs no gym interactions
with_gym = merged_properties_df[merged_properties_df['gym'] == 1]['total_interactions']
without_gym = merged_properties_df[merged_properties_df['gym'] == 0]['total_interactions']
t_stat, p_value = ttest_ind(with_gym, without_gym, equal_var=False)
hypothesis_result = "reject the null hypothesis" if p_value < 0.05 else "failed to reject the null hypothesis"
print("Q17: Hypothesis test result:", hypothesis_result)



Q17: Hypothesis test result: reject the null hypothesis


In [19]:
# QUESTION 18: Most frequent time category
def categorize_time(hour):
    if 0 <= hour < 6:
        return 'Midnight'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    else:
        return 'Evening'

property_interactions_df['hour'] = property_interactions_df['request_date'].dt.hour
property_interactions_df['time_category'] = property_interactions_df['hour'].apply(categorize_time)
most_frequent_time_category = property_interactions_df['time_category'].mode()[0]
print("Q18: Most frequent time category:", most_frequent_time_category)



Q18: Most frequent time category: Evening


In [20]:
# QUESTION 19: Most common activation date
most_common_activation_date = merged_properties_df['activation_date'].mode()[0]
print("Q19: Most common activation date:", most_common_activation_date)



Q19: Most common activation date: 2017-03-02 19:23:00


In [21]:
# QUESTION 20: Percentage of 'Anyone' lease type
anyone_lease_percentage = round(
    (merged_properties_df[merged_properties_df['lease_type'] == 'Anyone'].shape[0] / merged_properties_df.shape[0]) * 100,
    0
)
print("Q20: Percentage of properties under 'Anyone' lease type:", anyone_lease_percentage)


Q20: Percentage of properties under 'Anyone' lease type: 0.0
